# Yolov8 Model Training on nuImages


## Initialization

In [ ]:
#If this notebook is being run in Kaggle, run this to make sure that these files are not stored in the working directory

%cd ../../
%pwd

In [ ]:
%pip install wget
%pip install ultralytics
%pip install clearml

In [ ]:
import os
import wget
import tarfile
from ultralytics import YOLO
from clearml import Task

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY= # Insert your key
%env CLEARML_API_SECRET_KEY= # Insert your key
!wandb login #Insert key
clearml_project = 'nuimages-yolo'
clearml_task = 'train' 

In [ ]:
home_path = './' # make sure this is the same as in the dataset creation
dataset_path = os.path.join(home_path, '/datasets/nuImages') # make sure this is the same as in the dataset creation
yaml_path = os.path.join(dataset_path,'data.yaml') # make sure this is the same as in the dataset creation
raw_path = os.path.join(home_path, 'data/raw')
weights_path = os.path.join(home_path, 'weights')
if not os.path.exists(raw_path):
  os.makedirs(raw_path)
if not os.path.exists(dataset_path):
  os.makedirs(dataset_path)
if not os.path.exists(weights_path):
  os.makedirs(weights_path)

## Downlaoding and Extracting the Dataset.


### Download via Google Cloud

In [ ]:
links = {
    "nuImages-train-1.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-1.tar.gz",
    "nuImages-train-2.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-2.tar.gz",
    "nuImages-train-3.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-3.tar.gz",
    "nuImages-train-4.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-4.tar.gz",
    "nuImages-train-5.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-5.tar.gz",
    "nuImages-train-6.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-6.tar.gz",
    "nuImages-train-7.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-7.tar.gz",
    "nuImages-train-8.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-8.tar.gz",
    "nuImages-train-9.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-train-9.tar.gz",
    "nuImages-val-1.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-val-1.tar.gz",
    "nuImages-val-2.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-val-2.tar.gz",
    "nuImages-yaml.tar.gz": "https://storage.googleapis.com/nuimages/nuImages-yaml.tar.gz"
}

In [ ]:
for (archive_name, link) in links.items():
    wget.download(link, out=raw_path)
    with tarfile.open(os.path.join(raw_path, archive_name)) as archive:
        archive.extractall(path=home_path)
    os.remove(os.path.join(raw_path, archive_name))    

## Using Saved Weights

### Downloading Our Trained Weights

In [ ]:
wget.download('https://storage.googleapis.com/nuimages/best.pt', out=weights_path)
wget.download('https://storage.googleapis.com/nuimages/last.pt', out=weights_path)
ckpt = os.path.join(weights_path, 'last.pt')

### Using the default

In [ ]:
ckpt = 'yolov8l.pt'

## Training

In [ ]:
model = YOLO(ckpt)
results = model.train(data=yaml_path, project=clearml_project, name=clearml_task,
                        imgsz=960, epochs=4, warmup_epochs=0,
                        optimizer='AdamW', cos_lr=False, lr0=50*1e-6,
                        save=True, device=[0,1])

In [ ]:
# If in Kaggle, run this to obtain the results in the output folder

%cp -r /nuimages-yolo/ /kaggle/working/nuimages-yolo